In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import sys
import logging
import inspect
logging.basicConfig(level=logging.INFO)
print(sys.executable)

/home/peromoseq/anaconda3/envs/mmdeploy/bin/python3


### Get recording info (google sheets)

In [3]:
import requests
import pandas as pd
from io import BytesIO

In [4]:
# spreadsheet_url = 'https://docs.google.com/spreadsheet/ccc?key=14HIqUaSl_n-91hpAvmACY_iVY9nLKdlA6qklhxfZon0&output=csv&gid=0'
spreadsheet_url = "https://docs.google.com/spreadsheet/ccc?key=1jACsUmxuJ9Une59qmvzZGc1qXezKhKzD1zho2sEfcrU&output=csv&gid=0"
response = requests.get(spreadsheet_url)
recording_df = pd.read_csv(BytesIO(response.content))

In [5]:
recording_df[:3]

,Subject,duration_m,video_recording_id,ephys_id,calibration_id,video_location_on_o2,ephys_location_on_o2,calibration_location_on_o2,samplerate,username,n_ephys_streams,max_video_duration_m
0,M04002,10,24-05-01-13-26-43-110846,2024-05-01_13-26-37,24-05-01-13-45-07-825493,/n/groups/datta/tim_sainburg/datasets/chronic2...,/n/groups/datta/tim_sainburg/datasets/chronic2...,/n/groups/datta/tim_sainburg/datasets/chronic2...,150,tis697,1,10


### Submit job

In [6]:
from multicamera_airflow_pipeline.tim_240731.interface.local import LocalRunner
from pathlib import Path
from datetime import datetime

INFO:multicamera_airflow_pipeline.tim_240731.interface.local:Python interpreter binary location: /home/peromoseq/anaconda3/envs/mmdeploy/bin/python3


In [7]:
for idx, recording_row in recording_df.iterrows():
    break

### Job specific

In [8]:
import yaml
import textwrap
import time
import subprocess

In [9]:
# these are arguments
output_directory = Path("/n/groups/datta/tim_sainburg/datasets/scratch/") / "240808-3d-pipeline"
job_directory = Path('/n/groups/datta/tim_sainburg/datasets/scratch/jobs')
job_directory.mkdir(exist_ok=True, parents=True)
config_file = Path("/n/groups/datta/tim_sainburg/projects/multicamera_airflow_pipeline/multicamera_airflow_pipeline/tim_240731/default_config.yaml")

In [10]:
def get_gpu_memory_usage(gpu_id):
    """
    Returns the memory usage of a specified GPU.
    """
    result = subprocess.run(
        ["nvidia-smi", "--query-gpu=memory.used", "--format=csv,noheader,nounits", "-i", str(gpu_id)],
        capture_output=True,
        text=True
    )
    return int(result.stdout.strip())

def get_cuda_visible_device_with_lowest_memory():
    """
    Finds the GPU with the lowest memory usage and sets CUDA_VISIBLE_DEVICES to that GPU.
    """
    # Get the number of GPUs
    result = subprocess.run(
        ["nvidia-smi", "--query-gpu=name", "--format=csv,noheader"],
        capture_output=True,
        text=True
    )
    gpus = result.stdout.strip().split("\n")
    num_gpus = len(gpus)

    lowest_memory_usage = None
    best_gpu = None

    for i in range(num_gpus):
        mem_usage = get_gpu_memory_usage(i)
        if lowest_memory_usage is None or mem_usage < lowest_memory_usage:
            lowest_memory_usage = mem_usage
            best_gpu = i

    return best_gpu

In [11]:
def convert_minutes_to_hms(minutes_float):
    # Convert minutes to total seconds
    total_seconds = int(minutes_float * 60)
    
    # Extract hours, minutes, and seconds using divmod
    hours, remainder = divmod(total_seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    
    # Format as HH:MM:SS
    return f"{hours:02}:{minutes:02}:{seconds:02}"

In [12]:
def check_2d_completion(output_directory_predictions):
    return (output_directory_predictions / "completed.log").exists()

In [13]:
# load config
config_file = Path(config_file)
config = yaml.safe_load(open(config_file, "r"))

In [14]:
# where the video data is located
recording_directory = (
    Path(recording_row.video_location_on_o2) / recording_row.video_recording_id
)

In [15]:
# where to save output
output_directory_predictions = (
    output_directory / "2D_predictions" / recording_row.video_recording_id
)
output_directory_predictions.mkdir(parents=True, exist_ok=True)
current_datetime_str = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
current_job_directory = job_directory / current_datetime_str

# duration of video files (useful if videos are not properly muxxed)
expected_video_length_frames = recording_row.max_video_duration_m * 60 * recording_row.samplerate

# where tensorrt compiled models are saved (specific to GPU, so we compute on the fly)
tensorrt_model_directory = output_directory / "tensorrt" 

params = {
    "recording_directory": recording_directory.as_posix(),
    "output_directory_predictions": output_directory_predictions.as_posix(),
    "expected_video_length_frames": expected_video_length_frames,
    "tensorrt_model_directory": tensorrt_model_directory.as_posix(),
}

In [16]:
tensorrt_model_directory

PosixPath('/n/groups/datta/tim_sainburg/datasets/scratch/240808-3d-pipeline/tensorrt')

In [17]:
!ls {tensorrt_model_directory}

rtmdet_tiny_8xb32-300e_coco_chronic_24-05-04-17-51-58_216661
rtmpose-m_8xb64-210e_ap10k-256x256_24-05-04-21-35-13_305524


In [18]:
!ls {tensorrt_model_directory/ "rtmpose-m_8xb64-210e_ap10k-256x256_24-05-04-21-35-13_305524" / "NVIDIA_GeForce_RTX_4070_Ti"}

deploy.json  end2end.engine  output_pytorch.jpg   pipeline.json
detail.json  end2end.onnx    output_tensorrt.jpg


In [19]:
conda_env = Path(config["tensorrt_conversion_local"]["conda_env"])

In [20]:
conda_env

PosixPath('/home/peromoseq/anaconda3/envs/mmdeploy')

In [21]:
# Call the function
gpu_to_use = get_cuda_visible_device_with_lowest_memory()

print(gpu_to_use)

0


In [22]:
# create the job runner
runner = LocalRunner(
    job_name_prefix=f"{recording_row.video_recording_id}_2d_predictions",
    job_directory=current_job_directory,
    conda_env=conda_env,
    job_params=params,
    gpu_to_use = gpu_to_use,
)

In [23]:
if config["prediction_2d"]["use_tensorrt"]:
    runner.python_script = textwrap.dedent(
        f"""
    # load params
    import yaml
    params_file = "{runner.job_directory / f"{runner.job_name}.params.yaml"}"
    config_file = "{config_file.as_posix()}"

    params = yaml.safe_load(open(params_file, 'r'))
    config = yaml.safe_load(open(config_file, 'r'))
    
    config["tensorrt_conversion"]["conda_env"] = "{conda_env.as_posix()}"
    # convert models to tensorrt
    from multicamera_airflow_pipeline.tim_240731.keypoints.tensorrt import RTMModelConverter
    model_converter = RTMModelConverter(
        tensorrt_output_directory = params["tensorrt_model_directory"],
        is_local=True,
        **config["tensorrt_conversion"]
    )
    model_converter.run()
    
    # run predictions
    from multicamera_airflow_pipeline.tim_240731.keypoints.predict_2D import Inferencer2D
    camera_calibrator = Inferencer2D(
        recording_directory = params["recording_directory"],
        output_directory_predictions = params["output_directory_predictions"],
        expected_video_length_frames = params["expected_video_length_frames"],
        tensorrt_model_directory = params["tensorrt_model_directory"],
        **config["prediction_2d"]
    )
    camera_calibrator.run()
    """
    )
else:
    runner.python_script = textwrap.dedent(
        f"""
    # load params
    import yaml
    import sys
    print(sys.executable)
    params_file = "{runner.job_directory / f"{runner.job_name}.params.yaml"}"
    config_file = "{config_file.as_posix()}"

    params = yaml.safe_load(open(params_file, 'r'))
    config = yaml.safe_load(open(config_file, 'r'))

    # grab sync cameras function
    from multicamera_airflow_pipeline.tim_240731.keypoints.predict_2D import Inferencer2D
    camera_calibrator = Inferencer2D(
        recording_directory = params["recording_directory"],
        output_directory_predictions = params["output_directory_predictions"],
        expected_video_length_frames = params["expected_video_length_frames"],
        tensorrt_model_directory = params["tensorrt_model_directory"],
        **config["prediction_2d"]
    )
    camera_calibrator.run()
    """
    )

print(runner.python_script)


# load params
import yaml
params_file = "/n/groups/datta/tim_sainburg/datasets/scratch/jobs/20240813_152436_875670/24-05-01-13-26-43-110846_2d_predictions_24-08-13-2024-24-37-366679.params.yaml"
config_file = "/n/groups/datta/tim_sainburg/projects/multicamera_airflow_pipeline/multicamera_airflow_pipeline/tim_240731/default_config.yaml"

params = yaml.safe_load(open(params_file, 'r'))
config = yaml.safe_load(open(config_file, 'r'))

config["tensorrt_conversion"]["conda_env"] = "/home/peromoseq/anaconda3/envs/mmdeploy"
# convert models to tensorrt
from multicamera_airflow_pipeline.tim_240731.keypoints.tensorrt import RTMModelConverter
model_converter = RTMModelConverter(
    tensorrt_output_directory = params["tensorrt_model_directory"],
    is_local=True,
    **config["tensorrt_conversion"]
)
model_converter.run()

# run predictions
from multicamera_airflow_pipeline.tim_240731.keypoints.predict_2D import Inferencer2D
camera_calibrator = Inferencer2D(
    recording_directory = params["r

In [24]:
runner.run()

INFO:multicamera_airflow_pipeline.tim_240731.interface.local:Creating remote job directory: /n/groups/datta/tim_sainburg/datasets/scratch/jobs/20240813_152436_875670
INFO:multicamera_airflow_pipeline.tim_240731.interface.local:Writing job script, python script, params
INFO:multicamera_airflow_pipeline.tim_240731.interface.local:Using GPU 0
INFO:multicamera_airflow_pipeline.tim_240731.interface.local:Submitting job
INFO:multicamera_airflow_pipeline.tim_240731.interface.local:loading libmmdeploy_trt_net.so ...

INFO:multicamera_airflow_pipeline.tim_240731.interface.local:loading libmmdeploy_ort_net.so ...

